# Demo - Fairness Analysis of FICO

Adapted version of:

1. Fairness and machine learning book - [Chapter 2](https://fairmlbook.org/demographic.html) and [code repository](https://github.com/fairmlbook/fairmlbook.github.io).

2. Hardt, M., Price, E., & Srebro, N. (2016). [Equality of opportunity in supervised learning](https://arxiv.org/abs/1610.02413). In Advances in neural information processing systems (pp. 3315-3323).

3. [Attacking discrimination with smarter machine learning](https://research.google.com/bigpicture/attacking-discrimination-in-ml/) by Google

From [Wikipedia](https://en.wikipedia.org/wiki/Credit_score_in_the_United_States):
> Credit score in the United States is a number representing the creditworthiness of a person, the likelihood that person will pay his or her debts.  
Lenders, such as banks and credit card companies, use credit scores to evaluate the potential risk posed by lending money to consumers. Lenders allege that widespread use of credit scores has made credit more widely available and less expensive for many consumers


The analysis is based on data from [Report to the Congress on Credit Scoring and Its Effects on the Availability and Affordability of Credit](https://federalreserve.gov/boarddocs/rptcongress/creditscore/) by the Federal Reserve. The data set provides aggregate statistics from 2003 about a credit score, demographic information (race or ethnicity, gender, marital status), and outcomes (to be defined shortly). 


In the USA there are three major credit agencies, which are for-profit organizations. They offer risk score based on the data they collected. We are going to look into **FICO** score of TransUnion (called TransRisk). The TransRisk score is in turn based on a proprietary model created by FICO, hence often referred to as FICO scores. 


![Factors contributing to someone's credit score](https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Credit-score-chart.svg/640px-Credit-score-chart.svg.png)
<small>Source: Wikipedia</small>


From Fairness and Machine Learning - Limitations and Opportunities:
> Regulation of credit agencies in the United States started with the Fair Credit Reporting Act, first passed in 1970, that aims to promote the accuracy, fairness, and privacy of consumer of information collected by the reporting agencies. The Equal Credit Opportunity Act, a United States law enacted in 1974, makes it unlawful for any creditor to discriminate against any applicant the basis of race, color, religion, national origin, sex, marital status, or age.

### In our analysis we'll use on the joint statistics of score, race, and outcome.

In [ ]:
!pip install responsibly
import pandas as pd
import matplotlib.pylab as plt

from responsibly.fairness.metrics import plot_roc_curves
from responsibly.fairness.interventions.threshold import (find_thresholds,
                                                          plot_fpt_tpr,
                                                          plot_roc_curves_thresholds,
                                                          plot_costs,
                                                          plot_thresholds)

### FICO Dataset

FICO dataset can be loaded directly from `responsibly`. The dataset, in this case, is *aggregated*, i.e., there is no outcome and prediction information per individual, but summarized statistics for each FICO score and race/race/ethnicity group.

In [ ]:
from responsibly.dataset import build_FICO_dataset

FICO = build_FICO_dataset()

`FICO` is a dictionary that holds variaty of data:

In [ ]:
FICO.keys()

In [ ]:
help(build_FICO_dataset)

### Counts by Race or Ethnicity

In [ ]:
sum(FICO['totals'].values())

In [ ]:
pd.Series(FICO['totals']).plot(kind='barh', color=['b', 'r', 'g', 'y', 'k']);

### Score Distribution

The score used in the study is based on the TransUnion TransRisk score. TransUnion is a US credit-reporting agency. The TransRisk score is in turn based on a proprietary model created by FICO, hence often referred to as FICO scores. The Federal Reserve renormalized the scores for the study to vary from 0 to 100, with 0 being least creditworthy.

The information on race was provided by the Social Security Administration, thus relying on self-reported values.

The cumulative distribution of these credit scores strongly depends on the group as the next figure reveals.

In [ ]:
FICO['cdf'].head()

In [ ]:
FICO['cdf'].tail()

In [ ]:
f, ax = plt.subplots(1, figsize=(7, 5))

FICO['cdf'].plot(ax=ax, color=['b', 'r', 'g', 'y', 'k'])

plt.title('CDF by Group')
plt.ylabel('Cumulative Probability');

### Outcome Variable

**Performance variable** that measures a serious delinquency in at least one credit line of a certain time period:

> "(the) measure is based on the performance of new or existing accounts and measures whether individuals have been late 90 days or more on one or more of their accounts or had a public record item or a new collection agency account during the performance period." - *from the Federal Reserve report*

The `FICO['performance']` holds the percentage of *non-defaulters* for every score value (rows) and race/ethnicity group (columns):

In [ ]:
FICO['performance'].head()

In [ ]:
FICO['performance'].tail()

### Separation Fairness Criterion
By the separation criterion of a binary classifier, the *False Positive Rate (FPR)* and *True Positive Rate (TPR)* should be equal across the groups. The meaning of true positive rate is the rate of predicted positive performance given positive performance. Similarly, false positive rate is the rate of predicted negative performance given a positive performance.

Here, the TPR is the fraction of non-defaulters getting a loan, and the FPR is the fraction of defaulters getting a loan.

In [ ]:
plot_roc_curves(FICO['rocs'], FICO['aucs'],
                figsize=(7, 5));

In [ ]:
plot_roc_curves(FICO['rocs'], FICO['aucs'],
                figsize=(7, 5));

plt.xlim(0, 0.3)
plt.ylim(0.4, 1);

### Thresholds vs. FPR and TPR

The ROC is parameterized over the threshold, so the same threshold might be related to different (FPR, TPR) pairs for each group. We can observe it by plotting the FPR and the TPR as a function of the threshold by the groups.

In [ ]:
plot_fpt_tpr(FICO['rocs'], figsize=(15, 7),
             title_fontsize=15, text_fontsize=15);

Therefore, a naive choice of a single threshold will violate the separation fairness criterion, as there will be differences in FPR and TPR between the groups.

### Comparison of Different Criteria

* Single threshold (Group Unaware)
* Minimum Cost
* Independence (Demographic Parity, i.e. equal acceptance rate in all groups)
* FNR (Equality of opportunity)
* Separation (Equalized odds, i.e. equal TPR and FPR in all groups)

In making loans, the interest earned from a loan (the reward) is many times smaller than losing the loan amount during defaults (the cost). Assuming the cost is 5 times greater than the reward, we can define a cost matrix:
#### Cost: $FP = - 5 \cdot TP$

In [ ]:
COST_MATRIX = [[0, -5/6],
               [0,  1/6]]

In [ ]:
thresholds_data = find_thresholds(FICO['rocs'],
                                  FICO['proportions'],
                                  FICO['base_rate'],
                                  FICO['base_rates'],
                                  COST_MATRIX)

In [ ]:
plot_roc_curves_thresholds(FICO['rocs'], thresholds_data,
                           figsize=(7, 7),
                           title_fontsize=20, text_fontsize=15);

In [ ]:
plot_roc_curves_thresholds(FICO['rocs'], thresholds_data,
                           figsize=(7, 7),
                           title_fontsize=20, text_fontsize=15)

plt.xlim(0, 0.3)
plt.ylim(0.4, 1);

### Thresholds by Strategy and Group

In [ ]:
plot_thresholds(thresholds_data,
                xlim=(0, 100), figsize=(7, 7),
                title_fontsize=20, text_fontsize=15);

### Cost by Threshold Strategy

In [ ]:
plot_costs(thresholds_data);

### Sufficiency Fairness Criterion - Calibration

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))

FICO['performance'].plot(ax=axes[0])
axes[0].set_ylabel('Non-default rate')

for group in FICO['cdf'].columns:
    axes[1].plot(FICO['cdf'][group], FICO['performance'][group],
             label=group)
    
axes[1].set_ylabel('Non-default rate')
axes[1].set_xlabel('Score')
axes[1].legend();

Due to the differences in score distribution by group, it could nonetheless be the case that thresholding the score leads to a classifier with different positive predictive values in each group.